In [ ]:
import subprocess
import time
import csv
from datetime import datetime

# Function to collect Docker stats
def collect_docker_stats(container_name, duration=60, interval=1):
    stats = []
    cmd = f"docker stats {container_name} --no-stream --format '{{{{json .}}}}'"
    
    start_time = time.time()
    while time.time() - start_time < duration:
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        if result.returncode == 0:
            stat = result.stdout.strip()
            timestamp = datetime.now().isoformat()
            stats.append((timestamp, stat))
        else:
            print(f"Error collecting stats: {result.stderr}")
        time.sleep(interval)
    
    return stats

# Function to write stats to CSV
def write_stats_to_csv(stats, filename="docker_stats.csv"):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["timestamp", "cpu_usage", "memory_usage", "memory_limit", "memory_percent", "net_io", "block_io", "pids"])
        
        for timestamp, stat in stats:
            stat_dict = eval(stat)  # Convert JSON string to dictionary
            writer.writerow([
                timestamp,
                stat_dict.get("CPUPerc", ""),
                stat_dict.get("MemUsage", "").split('/')[0].strip(),
                stat_dict.get("MemUsage", "").split('/')[1].strip(),
                stat_dict.get("MemPerc", ""),
                stat_dict.get("NetIO", ""),
                stat_dict.get("BlockIO", ""),
                stat_dict.get("PIDs", "")
            ])

if __name__ == "__main__":
    container_name = "traffic-golang-server-1-1"  # Replace with your container name
    # container_name = "traffic-golang-server-2-a-1"  # Replace with your container name
    duration = 60  # Duration to collect stats (in seconds)
    interval = 1  # Interval between stats collection (in seconds)
    
    stats = collect_docker_stats(container_name, duration, interval)
    write_stats_to_csv(stats)
    print(f"Stats collected and written to docker_stats.csv")

Stats collected and written to docker_stats.csv
